<a href="https://colab.research.google.com/github/ZLY1223/Vaccine_LLM_Prompt_Engineering/blob/main/gemini_2.5_flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch
!pip install bitsandbytes
import torch
torch.cuda.empty_cache()
from transformers import set_seed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 41.0 MB/s eta 0:00:00


# M0: Demographics Only Model

In [ ]:
import pandas as pd
import re
import time
import google.generativeai as genai

genai.configure(api_key="xxxxxxx")

model = genai.GenerativeModel(model_name="gemini-2.5-flash")

seeds = list(range(1009, 1010))

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx"
df = pd.read_excel(file_path, index_col=None)

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]
        full_prompt = task_prompt
        response = model.generate_content(full_prompt)
        assistant_response = response.text.strip()

        print(f"Row {idx} Task 3 Response:\n{assistant_response}\n")

        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        option_mapping = {
            likelihood_vaccinated: "Vaccinated",
            likelihood_not_vaccinated: "Not Vaccinated"
        }

        max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
        selected_option = option_mapping[max_prob]

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(0.5)

    task_results_df = pd.DataFrame(task_results)
    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/gemini-2.5-flash/M0/Task_Results_Two_Options_M0_gemini2.5flash_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses saved to {task_output_path}")



==== Running for seed 1009 ====

Row 0 Task 3 Response:
Brief Reasoning: As a healthy 24-year-old, I don't feel the immediate necessity for the vaccine, believing my personal risk of severe illness from COVID-19 is low. I prefer to rely on my strong immune system and make personal health decisions.
Likelihood of Choosing Vaccinated: 35%
Likelihood of Choosing Not Vaccinated: 65%

Row 1 Task 3 Response:
Brief Reasoning: As a 27-year-old, getting vaccinated helps me stay healthy, avoid disrupting my work, and participate fully in social activities without unnecessary risks or restrictions. I prioritize practical health measures.
Likelihood of Choosing Vaccinated: 60%
Likelihood of Choosing Not Vaccinated: 40%



ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 730.83ms


Row 2 Task 3 Response:
Brief Reasoning: Given my age, good health, and financial stability, I don't feel the vaccine is highly necessary for my personal situation. I'd rather rely on my natural immunity and avoid any potential side effects.
Likelihood of Choosing Vaccinated: 35%
Likelihood of Choosing Not Vaccinated: 65%

Row 3 Task 3 Response:
Brief Reasoning: I generally approach health decisions with a degree of personal discretion. Given my good health and age, I'm not entirely convinced of the vaccine's necessity for my personal situation, and I have some lingering reservations about potential long-term effects.
Likelihood of Choosing Vaccinated: 45%
Likelihood of Choosing Not Vaccinated: 55%

Row 4 Task 3 Response:
Brief Reasoning: As a 36-year-old with a stable income, I would prioritize maintaining my health to avoid disruptions to my work and daily life, seeing vaccination as a practical step for protection. While not having a university degree, I would generally trust public 

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3070.72ms


Row 90 Task 3 Response:
Brief Reasoning: Being 68, I know I'm at a higher risk if I catch COVID-19. Getting the vaccine seems like the most straightforward way to protect myself and avoid serious health problems.
Likelihood of Choosing Vaccinated: 75%
Likelihood of Choosing Not Vaccinated: 25%

Row 91 Task 3 Response:
Brief Reasoning: As a 69-year-old, I am in a higher risk group for severe COVID-19 complications, so vaccination offers important protection for my health. I believe it's a sensible choice to reduce my chances of serious illness and hospitalization.
Likelihood of Choosing Vaccinated: 90%
Likelihood of Choosing Not Vaccinated: 10%

Row 92 Task 3 Response:
Brief Reasoning: As a 70-year-old, I'm well aware of the higher risks COVID-19 poses to people my age. Getting vaccinated is a sensible and practical way to protect my health and reduce the chances of severe illness.
Likelihood of Choosing Vaccinated: 90%
Likelihood of Choosing Not Vaccinated: 10%

Row 93 Task 3 Response:

# M1: Survey Model (Demogaphics + Prior Beliefs)

In [ ]:
import pandas as pd
import re
import time
import google.generativeai as genai

genai.configure(api_key="xxxxxxx")

model = genai.GenerativeModel(model_name="gemini-2.5-flash")
seeds = list(range(1000, 1010))

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M1.xlsx"
df = pd.read_excel(file_path, index_col=None)

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]
        full_prompt = task_prompt
        response = model.generate_content(full_prompt)
        assistant_response = response.text.strip()

        print(f"Row {idx} Task 3 Response:\n{assistant_response}\n")

        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        option_mapping = {
            likelihood_vaccinated: "Vaccinated",
            likelihood_not_vaccinated: "Not Vaccinated"
        }

        max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
        selected_option = option_mapping[max_prob]

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(1.5)

    task_results_df = pd.DataFrame(task_results)

    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/gemini-2.5-flash/M1/Task_Results_Two_Options_M1_gemini2.5flash_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses saved to {task_output_path}")


Streaming output truncated to the last 5000 lines.
Brief Reasoning: While I'm not entirely sure COVID-19 is a dangerous health threat, I believe the vaccine protects against Long-COVID. This, combined with my general trust in its safety and efficacy, makes me consider getting vaccinated for long-term health protection.
Likelihood of Choosing Vaccinated: 65%
Likelihood of Choosing Not Vaccinated: 35%

Row 1 Task 3 Response:
Brief Reasoning: I would choose to be vaccinated because I strongly believe COVID-19 is a dangerous health threat and trust that vaccines are safe, effective, and prevent the disease, including Long-COVID. My perception of the disease's risks far outweighs any vaccine concerns.
Likelihood of Choosing Vaccinated: 99%
Likelihood of Choosing Not Vaccinated: 1%

Row 2 Task 3 Response:
Brief Reasoning: Despite some lingering safety concerns, I strongly believe the risks of COVID-19 disease outweigh the risks of the vaccine, and I believe it protects against Long-COVID.
Li

# M2: Demographics + Media Diet

In [ ]:
import pandas as pd
import re
import time
import google.generativeai as genai

genai.configure(api_key="xxxxxxx")

model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# Load user sentences
df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/user_sentences_with_demographics_prior_beliefs.xlsx")

# Define fixed system prompt
system_prompt = "You are the person described in the profile below. Based on your demographic background and beliefs about COVID-19 and vaccines, your task is to choose which type of media sources you are most likely to follow for news."

# Define media options block
media_options_block = """There are four options you can choose from. Each consists of several representative media outlets and channels. Please pick the one that is mostly closely aligned with the person’s expressed beliefs, even if it does not match exactly:
1. The New York Times, The Guardian, MSNBC, DemocracyNow!, NPR
2. Fox News, National Review, The Federalist, New York Post, Washington Examiner
3. Reuters, BBC, The Wall Street Journal, Forbes, USA TODAY
4. Green Med Info, Children’s Health Defense, Mercola News, RFK Jr., The Epoch Times

Some individuals—particularly those who are even somewhat skeptical of elite institutions or who place value on national identity, personal freedom, or traditional viewpoints—may find themselves more aligned with Option 2.

You don’t need to fully agree with every source listed; if the overall tone or values resonate with you, that option may still be the best fit.
"""

# Define output instruction
output_format = """Please answer in the following format:
```
Answer: [1 / 2 / 3 / 4]
Reasoning: [Brief explanation based on the profile]
```
"""

# Store results
results = []

# Iterate over profiles
for i, row in df.iterrows():
    print(f"Processing user {i + 1}/{len(df)}")
    profile_text = row["user_sentence_no_occ"]
    full_prompt = f"{system_prompt}\n\n{media_options_block}\n\nNow imagine the following profile describes you:\n\n{profile_text}\n\nGiven this profile, which one of the four media options would you most likely follow?\n\n{output_format}"

    try:
            response = model.generate_content(full_prompt)
            assistant_response = response.text.strip()

            print(f"Seed {seed} - Row {idx} Response:\n{assistant_response}\n")
    except Exception as e:
        result_text = f"Error: {str(e)}"

    results.append({
        "id": row["id"] if "id" in row else i,
        "prompt": full_prompt,
        "response": assistant_response
    })

    time.sleep(1)  # avoid rate limits

results_df = pd.DataFrame(results)
results_df.to_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gemini2.5flash_demographics_beliefs_results_v4_four_options.xlsx", index=False)


Processing user 1/1000
Processing user 2/1000
Processing user 3/1000
Processing user 4/1000
Processing user 5/1000
Processing user 6/1000
Processing user 7/1000
Processing user 8/1000
Processing user 9/1000
Processing user 10/1000
Processing user 11/1000
Processing user 12/1000
Processing user 13/1000
Processing user 14/1000
Processing user 15/1000
Processing user 16/1000
Processing user 17/1000
Processing user 18/1000
Processing user 19/1000
Processing user 20/1000
Processing user 21/1000
Processing user 22/1000
Processing user 23/1000
Processing user 24/1000
Processing user 25/1000
Processing user 26/1000
Processing user 27/1000
Processing user 28/1000
Processing user 29/1000
Processing user 30/1000
Processing user 31/1000
Processing user 32/1000
Processing user 33/1000
Processing user 34/1000
Processing user 35/1000
Processing user 36/1000
Processing user 37/1000
Processing user 38/1000
Processing user 39/1000
Processing user 40/1000
Processing user 41/1000
Processing user 42/1000
P

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 13454.37ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 12561.44ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 6500.48ms


Processing user 262/1000
Processing user 263/1000
Processing user 264/1000
Processing user 265/1000
Processing user 266/1000
Processing user 267/1000
Processing user 268/1000
Processing user 269/1000
Processing user 270/1000
Processing user 271/1000
Processing user 272/1000
Processing user 273/1000
Processing user 274/1000
Processing user 275/1000
Processing user 276/1000
Processing user 277/1000
Processing user 278/1000
Processing user 279/1000
Processing user 280/1000
Processing user 281/1000
Processing user 282/1000
Processing user 283/1000
Processing user 284/1000
Processing user 285/1000
Processing user 286/1000
Processing user 287/1000
Processing user 288/1000
Processing user 289/1000
Processing user 290/1000
Processing user 291/1000
Processing user 292/1000
Processing user 293/1000
Processing user 294/1000
Processing user 295/1000
Processing user 296/1000
Processing user 297/1000
Processing user 298/1000
Processing user 299/1000
Processing user 300/1000
Processing user 301/1000


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2485.16ms


Processing user 365/1000
Processing user 366/1000
Processing user 367/1000
Processing user 368/1000
Processing user 369/1000
Processing user 370/1000
Processing user 371/1000
Processing user 372/1000
Processing user 373/1000
Processing user 374/1000
Processing user 375/1000
Processing user 376/1000
Processing user 377/1000
Processing user 378/1000
Processing user 379/1000
Processing user 380/1000
Processing user 381/1000
Processing user 382/1000
Processing user 383/1000
Processing user 384/1000
Processing user 385/1000
Processing user 386/1000
Processing user 387/1000
Processing user 388/1000
Processing user 389/1000
Processing user 390/1000
Processing user 391/1000
Processing user 392/1000
Processing user 393/1000
Processing user 394/1000
Processing user 395/1000
Processing user 396/1000
Processing user 397/1000
Processing user 398/1000
Processing user 399/1000
Processing user 400/1000
Processing user 401/1000
Processing user 402/1000
Processing user 403/1000
Processing user 404/1000


...

Processing user 572/1000
Processing user 573/1000
Processing user 574/1000
Processing user 575/1000
Processing user 576/1000
Processing user 577/1000
Processing user 578/1000
Processing user 579/1000
Processing user 580/1000
Processing user 581/1000
Processing user 582/1000
Processing user 583/1000
Processing user 584/1000
Processing user 585/1000
Processing user 586/1000
Processing user 587/1000
Processing user 588/1000
Processing user 589/1000
Processing user 590/1000
Processing user 591/1000
Processing user 592/1000
Processing user 593/1000
Processing user 594/1000
Processing user 595/1000
Processing user 596/1000
Processing user 597/1000
Processing user 598/1000
Processing user 599/1000
Processing user 600/1000
Processing user 601/1000
Processing user 602/1000
Processing user 603/1000
Processing user 604/1000
Processing user 605/1000
Processing user 606/1000
Processing user 607/1000
Processing user 608/1000
Processing user 609/1000
Processing user 610/1000
Processing user 611/1000


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2178.96ms


Processing user 937/1000
Processing user 938/1000
Processing user 939/1000
Processing user 940/1000
Processing user 941/1000
Processing user 942/1000
Processing user 943/1000
Processing user 944/1000
Processing user 945/1000
Processing user 946/1000
Processing user 947/1000
Processing user 948/1000


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3682.13ms


Processing user 949/1000
Processing user 950/1000
Processing user 951/1000
Processing user 952/1000
Processing user 953/1000
Processing user 954/1000
Processing user 955/1000
Processing user 956/1000
Processing user 957/1000
Processing user 958/1000
Processing user 959/1000
Processing user 960/1000
Processing user 961/1000
Processing user 962/1000
Processing user 963/1000
Processing user 964/1000
Processing user 965/1000
Processing user 966/1000
Processing user 967/1000
Processing user 968/1000
Processing user 969/1000
Processing user 970/1000
Processing user 971/1000
Processing user 972/1000
Processing user 973/1000
Processing user 974/1000
Processing user 975/1000
Processing user 976/1000
Processing user 977/1000
Processing user 978/1000
Processing user 979/1000
Processing user 980/1000
Processing user 981/1000
Processing user 982/1000
Processing user 983/1000
Processing user 984/1000
Processing user 985/1000
Processing user 986/1000
Processing user 987/1000
Processing user 988/1000


In [ ]:
import re
import pandas as pd

results_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gemini2.5flash_demographics_beliefs_results_v4_four_options.xlsx")
def extract_answer(text):
    match = re.search(r'Answer:\s*(\d)', str(text))
    return int(match.group(1)) if match else None

results_df['predicted_answer'] = results_df['response'].apply(extract_answer)

label_map = {
    1: "Left Echochamber",
    2: "Right Echochamber",
    3: "Center-ish",
    4: "Misinformation Only"
}

results_df['media_diet_label'] = results_df['predicted_answer'].map(label_map)
print("Gemini-2.5-flash Frequency distribution:")
print(results_df['media_diet_label'].value_counts().sort_index())

Gemini-2.5-flash Frequency distribution:
media_diet_label
Center-ish             315
Left Echochamber       322
Misinformation Only    252
Right Echochamber      111
Name: count, dtype: int64


In [ ]:
results_df.to_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gemini2.5flash_demographics_beliefs_results_v4_four_options.xlsx", index=False)

In [ ]:
import pandas as pd
import random
import re

task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
mapping_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gemini2.5flash_demographics_beliefs_results_v4_four_options.xlsx")
media_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/media_diet_with_extracted_words_edited.xlsx")
matrix_df = pd.read_excel(
    "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/Media_Diets.xlsx",
    sheet_name="Media Diets",
    index_col=0
)

mapping_df["id"] += 1
matrix_df = matrix_df.rename(columns={
    "Misinfo": "Low Credibility",
})

task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], left_on="id", right_on="id", how="left")

diet_to_article_quota = {
    diet: matrix_df.loc[diet][matrix_df.loc[diet] > 0].to_dict()
    for diet in matrix_df.index
}

def build_rag_prompt(task_prompt, diet, _):
    quota = diet_to_article_quota.get(diet, {})
    selected_articles = []

    for cat, count in quota.items():
        available = media_df[media_df["Category"] == cat][["Headline", "First_100_Words"]].dropna()
        if len(available) == 0:
            print(f"[Warning] No articles available for category '{cat}' (diet: {diet})")
            continue
        samples = available.sample(n=min(count, len(available)), random_state=random.randint(0, 10000))
        selected_articles.extend(list(zip(samples["Headline"], samples["First_100_Words"])))

    random.shuffle(selected_articles)

    disease_block = f"""% Media Diet Information
Assume today is June 29, 2022. Your decision about receiving a COVID-19 vaccine is primarily shaped by your demographics and prior beliefs. However, the kind of information you are exposed to in the media may also influence your thinking.
Below are five real-world news articles that reflect the type of media you are likely to encounter.
As you read them, consider:
- Do any of these articles reinforce your existing beliefs?
- Do any contradict them or introduce new concerns or reassurances?
Unless strongly challenged by the articles, your decision should remain aligned with your prior beliefs. Only adjust your position if a particular article clearly shifts your perception.
If one article is especially influential, briefly mention it in your reasoning.
"""

    media_context = "The articles are listed below:\n"
    for i, (title, content) in enumerate(selected_articles[:5]):
        media_context += f"Article {i+1}: {title.strip()}\n{content.strip()}\n"

    insert_point = task_prompt.find("% Task Prompt")
    return task_prompt if insert_point == -1 else (
        task_prompt[:insert_point] + disease_block + media_context + task_prompt[insert_point:]
    )

def safe_build(row):
    diet = row["media_diet_label"]
    if diet not in diet_to_article_quota:
        print(f"[Warning] Diet not found in quota matrix: {diet}")
    return build_rag_prompt(row["Task_Prompt"], diet, None)

for i in range(10):
    random.seed(1000 + i)

    task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
    task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], on="id", how="left")

    task_df["Task_Prompt_Full"] = task_df.apply(safe_build, axis=1)

    task_df["Task_Prompt_Full"] = task_df["Task_Prompt_Full"].apply(
        lambda text: re.sub(
            r"Now, based on the information above,",
            "Now, based on your demographics and the content of the five articles above,",
            text
        )
    )

    output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed{1000+i}.xlsx"
    task_df[["id", "Task_Prompt_Full", "media_diet_label"]].to_excel(output_path, index=False)
    print(f"[Saved] Seed {1000 + i} -> {output_path}")

[Saved] Seed 1000 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed1000.xlsx
[Saved] Seed 1001 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed1001.xlsx
[Saved] Seed 1002 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed1002.xlsx
[Saved] Seed 1003 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed1003.xlsx
[Saved] Seed 1004 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_four_diets_M2_gemini2.5flash_seed1004.xlsx
[Saved] Seed 1005 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs

In [ ]:
import pandas as pd
import re
import time
import google.generativeai as genai

genai.configure(api_key="xxxxxxx")

model = genai.GenerativeModel(model_name="gemini-2.5-flash")

input_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/"
output_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/gemini-2.5-flash/M2/"

seed=42
seeds_to_run = list(range(1000, 1010))

for seed in seeds_to_run:
    input_file = f"{input_base}task_inputs_two_options_four_diets_M2_gemini2.5flash_seed{seed}.xlsx"
    output_file = f"{output_base}Task_Results_Two_Options_Four_Diets_M2_gemini2.5flash_seed{seed}.xlsx"

    try:
        df = pd.read_excel(input_file)
    except Exception as e:
        print(f"Failed to read {input_file}: {e}")
        continue

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt_Full"]

        try:
            full_prompt = task_prompt
            response = model.generate_content(full_prompt)
            assistant_response = response.text.strip()

            print(f"Seed {seed} - Row {idx} Response:\n{assistant_response}\n")

            match_vaccinated = re.search(r"[Ll]ikelihood of Choosing Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)
            match_not_vaccinated = re.search(r"[Ll]ikelihood of Choosing Not Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)

            l_vac = int(match_vaccinated.group(1)) if match_vaccinated else -1
            l_not = int(match_not_vaccinated.group(1)) if match_not_vaccinated else -1

            if l_vac == -1 or l_not == -1:
                decision = "Extraction Failed"
            else:
                decision = "Vaccinated" if l_vac >= l_not else "Not Vaccinated"

            task_results.append({
                "id": id,
                "Task_Prompt": task_prompt,
                "Task_Response": assistant_response,
                "Vaccination Decision": decision,
                "Likelihood of Choosing Vaccinated (%)": f"{l_vac}%" if l_vac >= 0 else "NA",
                "Likelihood of Choosing Not Vaccinated (%)": f"{l_not}%" if l_not >= 0 else "NA"
            })

        except Exception as e:
            print(f"Error on seed {seed}, row {idx}: {e}")
            time.sleep(1.5)

        time.sleep(0.5)

    df_results = pd.DataFrame(task_results)
    df_results.to_excel(output_file, index=False)
    print(f"Seed {seed} finished. Results saved to {output_file}\n")


Seed 1006 - Row 0 Response:
Brief Reasoning: The articles reinforce the idea that COVID-19 is manageable for healthy individuals and that governmental overreach is a concern, as highlighted by Sen. Rand Paul. The perceived diminishing severity and the continuous need for new variant-specific vaccines also contribute to not prioritizing vaccination for myself.
Likelihood of Choosing Vaccinated: 15%
Likelihood of Choosing Not Vaccinated: 85%

Seed 1006 - Row 1 Response:
Brief Reasoning: The recurring need for boosters mentioned by the Pfizer CEO (Article 3) and the fact that vaccinated individuals can still contract COVID-19 (Article 4) make me question the long-term effectiveness and necessity of the vaccine for my demographic, despite concerns about Long Covid. I prefer to make my own health choices as society moves towards living with the virus.
Likelihood of Choosing Vaccinated: 40%
Likelihood of Choosing Not Vaccinated: 60%

Seed 1006 - Row 2 Response:
Brief Reasoning: Article 3 sig

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3286.66ms


Streaming output truncated to the last 5000 lines.
Likelihood of Choosing Vaccinated: 10%
Likelihood of Choosing Not Vaccinated: 90%

Seed 1007 - Row 154 Response:
Brief Reasoning: Article 5, highlighting that even mild COVID can cause brain shrinkage and cognitive decline, is a significant new concern for my long-term health and capabilities. This potential for permanent repercussions makes getting vaccinated a prudent step to reduce my risk of infection and its severe effects.
Likelihood of Choosing Vaccinated: 90%
Likelihood of Choosing Not Vaccinated: 10%

Seed 1007 - Row 155 Response:
Brief Reasoning: The articles, particularly the one highlighting that "high-tech pandemic solutions don't save lives" but rather "boost profits," reinforce my existing skepticism towards official COVID-19 responses and the motivations behind them. The perceived confusion from authorities and validation of personal side effects further solidify my decision not to get vaccinated.
Likelihood of Choosing